# Bidding

This is a tutorial for how to use the bidding engine

In [1]:
import sys
import os
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotBid
from util import hand_to_str
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)

models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

Instructions for updating:
non-resource variables are not supported in the long term
Using old version of models
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo_same-5556000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/Jack/lr3-1000000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-32768000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/righty_nt-475000
INFO:tensorflow:Restoring parameters from ..\Models/decl_nt-950000
INFO:tensorflow:Restoring parameters from ..\Models/lefty_suit-999000
INFO:tensorflow:Restoring parameters from ..\Models/dummy_suit-999000
INFO:tensorflow:Restoring parameters

### Running through an example

In [2]:
# East deals, EW vulnerable.
vuln_ns, vuln_ew = False, False

# you sit West and hold:
hand = 'J.J82.A98632.T52'

# the auction goes:
# (a few words about 'PAD_START':
# the auction is padded to dealer North
# if North is not dealer, than we have to put in a 'PAD_START' for every seat that was skipped
# if East deals we have one pad (because North is skipped)
# if South deals we have two pads (because North and East are skipped)
# etc.)
auction = ["1H", "1N", "2S"]
bot_bid = BotBid([vuln_ns, vuln_ew], hand, models, sampler, seat=3, dealer=2, verbose=False)

bid = bot_bid.bid(auction)
bid.to_dict()['candidates']

# what's your bid?

2
3
['1H', '1N', '2S']
get_bid_candidates ['1H', '1N', '2S']
LHO 1H
PAR 1N
RHO 2S


Loaded lib dds.dll


[{'call': '3D', 'insta_score': 0.388, 'expected_score': 254, 'adjustment': 19},
 {'call': 'PASS', 'insta_score': 0.453, 'expected_score': 4, 'adjustment': 23}]

In [3]:
bid.samples

['8x.AQxxx.QJx.Kxx AKx.KTxx.Kx.Q8xx QT9xxxx.9.Tx.AJ9 J.J8x.A98xxx.Txx 0.84331 \n 1H-1N-2S-P-P-P (9, 8.8, 8.6)  \n 1H-1N-2S-3D-P-P-3H-3N-P-P-P (5, 6.3, 6.4) ',
 'Axx.AKT9xx.x.J9x Kx.Qxx.KQJT.KQxx QT98xxx.x.xx.A8x J.J8x.A98xxx.Txx 0.81571 \n 1H-1N-2S-P-P-P (11, 10.0, 10.5)  \n 1H-1N-2S-3D-P-P-4H-X-4S-P-P-X-P-P-P (11, 10.0, 10.6) ',
 'Ax.ATxxx.Tx.AJ8x Qxx.KQ9x.KQxx.Kx KT98xxx.x.J.Q9xx J.J8x.A98xxx.Txx 0.81217 \n 1H-1N-2S-P-P-P (11, 10.9, 10.2)  \n 1H-1N-2S-3D-P-P-4H-X-4S-P-P-X-P-P-P (11, 10.9, 10.1) ',
 'Tx.KQT9x.Kx.QJxx AQ8.Axxx.Qx.A98x K9xxxxx.x.JTx.Kx J.J8x.A98xxx.Txx 0.81157 \n 1H-1N-2S-P-P-P (8, 8.0, 7.9)  \n 1H-1N-2S-3D-P-P-4H-X-4S-P-P-X-P-P-P (8, 8.0, 7.8) ',
 'x.ATxxxx.Qxx.KQx AQ8x.KQ9x.Tx.AJx KT9xxxx..KJ.98xx J.J8x.A98xxx.Txx 0.79415 \n 1H-1N-2S-P-P-P (7, 7.9, 7.9)  \n 1H-1N-2S-3D-P-P-3H-3N-P-P-4H-X-P-P-P (5, 5.5, 6.0) ',
 'ATx.ATxxxx.Qx.xx K9.KQ9.KTx.AQ98x Q8xxxxx.x.Jx.KJx J.J8x.A98xxx.Txx 0.76204 \n 1H-1N-2S-P-P-P (9, 9.1, 9.1)  \n 1H-1N-2S-3D-P-P-3H-3N-P-P-4H-X-P-P-P (6, 5.9, 

In [5]:
auction = ["1H", "1N", "2S","3D","3S","PASS","4S"]
bid = bot_bid.bid(auction)

bid.to_dict()['candidates']

2
3
['1H', '1N', '2S', '3D', '3S', 'PASS', '4S']
get_bid_candidates ['1H', '1N', '2S', '3D', '3S', 'PASS', '4S']
LHO 1H
PAR 1N
RHO 2S
Me 3D
LHO 3S
PAR PASS
RHO 4S


[{'call': 'PASS',
  'insta_score': 0.628,
  'expected_score': -194,
  'adjustment': 314},
 {'call': 'X',
  'insta_score': 0.228,
  'expected_score': -275,
  'adjustment': 114},
 {'call': '5D',
  'insta_score': 0.143,
  'expected_score': -590,
  'adjustment': 71}]

in the output above:
- `insta_score` reflects what the neural network prefers to bid
- `expected_score` is based on what you expect to score on difference samples (for example, 70% of the time you are making and 30% of the time you are going one off, then the expected score is 0.7 * 620 - 0.3 * 100)

### Samples consistent with the auction so far

Above we said that we are computig expected scores on samples consistent with the auction so far.

How do we get these samples?

We get some samples in the response we get from the bot (i.e `bid.samples`)

In [6]:
bid.samples

['Q9x.QT9xx.Tx.AK8 Kxx.AKxx.Kxx.Qxx AT8xxx.x.QJ.J9xx J.J8x.A98xxx.Txx 0.54403 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (9, 9.2, 9.2)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (9, 9.2, 9.2)  \n 1H-1N-2S-3D-3S-P-4S-5D-P-P-X-P-P-P (9, 8.5, 8.4) ',
 'AQTx.9xxxx..AQ8x xxx.AKTx.KQx.K9x K98xx.Q.JTxx.Jxx J.J8x.A98xxx.Txx 0.53757 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (10, 10.3, 10.3)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (10, 10.3, 10.1)  \n 1H-1N-2S-3D-3S-P-4S-5D-X-P-P-P (9, 8.3, 9.7) ',
 'A9x.QT9xxx.J.KQ9 KTx.AKxx.KQT.xxx Q8xxxx..xxx.AJ8x J.J8x.A98xxx.Txx 0.53577 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (11, 10.8, 9.9)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (11, 10.8, 9.6)  \n 1H-1N-2S-3D-3S-P-4S-5D-P-P-X-P-P-P (6, 7.6, 8.6) ',
 'A9xx.KQTxx.Qxx.Q KQx.A9xx.KJx.K9x T8xxx.x.T.AJ8xxx J.J8x.A98xxx.Txx 0.51854 \n 1H-1N-2S-3D-3S-P-4S-P-P-P (9, 9.1, 9.1)  \n 1H-1N-2S-3D-3S-P-4S-X-P-P-P (9, 9.1, 8.9)  \n 1H-1N-2S-3D-3S-P-4S-5D-P-P-X-P-P-P (9, 8.0, 8.7) ',
 'AQ8x.KQxxx.x.Q8x Kxx.ATx.KJT.A9xx T9xxx.9x.Qxx.KJx J.J8x.A98xxx.Txx 0.50689 \n 1H-1N-2S-3D-3S-P-4S-P